In [ ]:
import sys, sqlite3
import pandas as pd
from PyQt5.QtWidgets import QApplication, QMainWindow, QWidget, QGridLayout, QLabel, QPushButton, \
    QComboBox, QLineEdit
from PyQt5.QtCore import QTimer, Qt
from PyQt5.QtGui import QIcon
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.figure import Figure

connect = sqlite3.connect('data.db')
s = pd.read_sql('select sql from sqlite_master where name=="T"', connect)['sql'][0]
cols, state = str(len(list( filter(lambda x:'data' in x,s.split(',')) ))), '1'
data = { 'data'+str(i):[0]*100 for i in range(1,1+int(cols)) }
pred = { 'pred'+str(i):[1]*100 for i in range(1,1+int(cols)) }

class MainWindow(QMainWindow):
    def __init__(self):
        global state, cols
        super().__init__()
        self.setWindowTitle("Real time edge device status")
        self.setGeometry(150, 175, 1500, 800)
        self.setWindowIcon(QIcon("icon.png"))
        self.setStyleSheet("QMainWindow{background:qlineargradient(spread:\
            pad,x1:0,y1:0,x2:0,y2:1, stop:0 #003377, stop:1 #0000AA)}")
        
        title = QLabel(self)
        title.setText('MiAION-SeeR 異常檢測')
        title.setStyleSheet("color:white; font-family:Microsoft JhengHei; font-size:48px;")
        
        dataFig = PlotCanvas(self, width=5, height=3, title='data', color='#ff0000')
        self.dataFigThread = QTimer(dataFig)
        self.dataFigThread.timeout.connect(dataFig.plotc)
        self.dataFigThread.start(1000)
        
        predFig = PlotCanvas(self, width=5, height=3, title='pred', color='#00ff00')
        self.predFigThread = QTimer(predFig)
        self.predFigThread.timeout.connect(predFig.plotc)
        self.predFigThread.start(1000)
        
        prevButton = QPushButton(self)
        prevButton.setText('prev')
        prevButton.setStyleSheet("font-family:Microsoft JhengHei; font-size:24px;")
        prevButton.clicked.connect(self.prevAction)
        
        self.page = QLabel(self)
        self.page.setText('Page: '+state+'/'+cols)
        self.page.setStyleSheet("color:white; font-family:Microsoft JhengHei; font-size:24px;")
        self.page.setAlignment(Qt.AlignCenter)
        
        nextButton = QPushButton(self)
        nextButton.setText('next')
        nextButton.setStyleSheet("font-family:Microsoft JhengHei; font-size:24px;")
        nextButton.clicked.connect(self.nextAction)
        
        self.CPU = QLabel(self)
        self.CPU.setText('CPU: -1')
        self.CPU.setStyleSheet("font-family:Microsoft JhengHei; font-size:24px; background-color:\
            orange; border-radius:12px;")
        
        self.memory = QLabel(self)
        self.memory.setText('memory: -1')
        self.memory.setStyleSheet("font-family:Microsoft JhengHei; font-size:24px; background-color:\
            green; border-radius:12px;")
        
        self.temperature = QLabel(self)
        self.temperature.setText('CPU: -1')
        self.temperature.setStyleSheet("font-family:Microsoft JhengHei; font-size:24px; \
            background-color:white; border-radius:12px;")
        
        self.accessDBThread = QTimer(self)
        self.accessDBThread.timeout.connect(self.accessDB)
        self.accessDBThread.start(1000)
        
        retrainDataText = QLabel(self)
        retrainDataText.setText('Retrain parameter: ')
        retrainDataText.setStyleSheet("font-family:Microsoft JhengHei; font-size:24px; color:white")
        
        retrainComboBox = QComboBox(self)
        retrainComboBox.addItems( ['data'+str(i) for i in range(1,int(cols)+1)] )
        retrainComboBox.setStyleSheet("font-family:Microsoft JhengHei; font-size:24px;")
        
        retrainNumberText = QLabel(self)
        retrainNumberText.setText('Retrain data numbers: ')
        retrainNumberText.setStyleSheet("font-family:Microsoft JhengHei; font-size:24px; color:white")
        
        retrainDataNumber = QLineEdit(self)
        retrainDataNumber.setText('30000')
        retrainDataNumber.setStyleSheet("font-family:Microsoft JhengHei; font-size:24px;")

        retrainButton = QPushButton(self)
        retrainButton.setText('Start Retrain !!!')
        retrainButton.setStyleSheet("font-family:Microsoft JhengHei; font-size:24px;")
        
        self.log = QLabel(self)
        self.log.setText('This is log')
        self.log.setStyleSheet("font-family:Microsoft JhengHei; font-size:24px; background-color:white")
        
        layout = QGridLayout()
        layout.addWidget(title, 0, 0, 1, 8, Qt.AlignCenter) # merge table from 0,0 thick1 width4
        layout.addWidget(dataFig, 1, 0, 3, 3, Qt.AlignCenter)
        layout.addWidget(predFig, 1, 4, 3, 3, Qt.AlignCenter)
        layout.addWidget(prevButton, 4, 0)
        layout.addWidget(self.page, 4, 1)
        layout.addWidget(nextButton, 4, 2)
        layout.addWidget(self.CPU, 5, 0)
        layout.addWidget(self.memory, 5, 1)
        layout.addWidget(self.temperature, 5, 2)
        layout.addWidget(self.log, 4, 3, 2, 1, Qt.AlignCenter)
        layout.addWidget(retrainDataText, 4, 4)
        layout.addWidget(retrainComboBox, 4, 5)
        layout.addWidget(retrainNumberText, 5, 4)
        layout.addWidget(retrainDataNumber, 5, 5)
        layout.addWidget(retrainButton, 4, 7, 2, 1, Qt.AlignCenter)
        
        widget = QWidget() # available resize and move
        widget.setLayout(layout)
        self.setCentralWidget(widget)
    
    def prevAction(self):
        global state, cols
        if state!='1':
            state = str(int(state)-1)
            self.page.setText('Page: '+state+'/'+cols)
    
    def nextAction(self):
        global state, cols
        if state!=cols:
            state = str(int(state)+1)
            self.page.setText('Page: '+state+'/'+cols)
    
    def accessDB(self):
        global data, pred, hardware, cols, connect
        df = pd.read_sql('select* from T order by time desc limit 1', connect)
        for i in range(1,int(cols)+1):
            data['data'+str(i)] = data['data'+str(i)][1:] + [df['data'+str(i)][0]]
            pred['pred'+str(i)] = pred['pred'+str(i)][1:] + [df['pred'+str(i)][0]]
        self.CPU.setText('CPU: '+str(df['CPU'][0]))
        self.memory.setText('memory: '+str(df['memory'][0]))
        self.temperature.setText('temperature: '+str(df['temperature'][0]))
        
class PlotCanvas(FigureCanvas):
    def __init__(self, parent=None, width=5, height=3, title='?', color='b'):
        super().__init__(Figure(figsize=(width, height), dpi=100, tight_layout={"pad":0.4,\
            "w_pad":0.5, "h_pad":1.0})) # self.figure is the object that plt.figure() returns
        self.title, self.color = title, color
        self.setParent(parent)
        self.plotc()

    def plotc(self):
        global state
        self.figure.patch.set_facecolor('black')   # border
        ax = self.figure.add_subplot(111)
        ax.clear()
        ax.axes.set_facecolor('black')             # inner
        ax.set_title(self.title+state, color='white', fontsize=16)
        ax.spines['bottom'].set_color('white')     # spines
        ax.tick_params(axis='x', colors='white')   # ticks
        ax.spines['left'].set_color('white')
        ax.tick_params(axis='y', colors='white')
        if self.title[:4]=='data':
            global data
            ax.plot(data['data'+state], self.color)
        else:
            global pred
            ax.plot(pred['pred'+state], self.color)
        self.draw()
    
app = QApplication(sys.argv)
MainWindow = MainWindow()
MainWindow.show()
sys.exit(app.exec_())

c:\users\bnbsking\miniconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:152: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.


In [ ]:
# QLabel, QPushButton, QLineEdit (Qwidget)
# integrate by layout and transfer to a widget W
# self.setCentralWidget(W)

In [ ]:
#ax.xaxis.label.set_color('white')
#ax.yaxis.label.set_color('white')
#ax.set_xlabel('time')
#ax.set_ylim(ylb,yub)

In [ ]:
s='CREATE TABLE "T" (data1 FLOAT, data2 FLOAT, data3 FLOAT, pred1 FLOAT, pred2 FLOAT, \
	pred3 FLOAT, \
	"CPU" FLOAT, \
	memory FLOAT, \
	temperature FLOAT, time FLOATt\
)'

list(filter(lambda x:'data' in x,s.split(',')))

In [ ]:
df = pd.read_sql('select* from T order by time desc limit 1', connect)
df

In [ ]:
df = pd.read_sql('select* from T order by time desc', connect)
df[df['time']!=0]